# 1. Trigger Giriş

* SQL Serverin database objelerinden biridir.
* İngilizce anlamı "Tetikleyici" demektir.
* Veritabanı tablosunda bir işlem gerçekleştiğinde başka bir işlemin otomatik olarak gerçekleşmesi anlamına gelir.
* Burada işlem olarak kastedilen data manipülasyonudur.
* Data manipülasyonları ise Insert, Update ve Delete işlemleridir.
* Yazılan triggerlar Insert, Update ve Delete işlemlerinden sonra otomatik olarak çalışan yapılardır.
    * **Bir kayıt ile alakalı bir değişiklik olması durumudur.**
* Triggerların içinde sanal olarak oluşan Inserted ve Deleted tabloları vardır.
    * Inserted tablosu yeni eklenen kaydın ya da update edildiğinde değişen kaydın yeni değerini tutar.
    * Deleted tablosu ise silinen kaydı ya da değiştirilen kaydın eski değerini tutar.
* Triggerlar genelde otomatik toplam hesaplama, son değeri alma ya da loglama amacıyla kullanılır.

### Trigger ile Toplam Hesaplama

Diyelim bir depon var ve bu depoda sürekli bir malzeme sürkilasyonu var. 10000 kalem malzemen olduğunu düşün, gün içerisinde bu malzemeler sürekli gidiyor geliyor, bir hareket söz konusu. Hangi üründen elinde ne kadar kaldığını bilmek istiyorsun. Gelen ve gidenleri toplayıp çıkararak stok toplamlarını bulmaya çalışıyorsun.

<img src = "https://i.hizliresim.com/eka03x8.png">

Üç farklı ürünün olduğunu düşün. Stok hareketlerine bakarak hangi tarihte ne kadar ve ne işlem yapıldığını görebiliyorsun. Bu hareketlerin çok fazla olacağını düşünürsek her seferinde tek tek işlemleri hesaplamak yerine elde edilen miktarları stok toplamları şeklinde her işlem sonrası güncellemek daha iyi performans sağlar. Stok toplamlarında bu işlemler sadece farklı ürün kadar yer tutar. SQL Serverda bu işlem triggerlar ile gerçekleştirilebilir.

### Trigger ile Kartlı Geçiş Sistemi Örneği

<img src = "https://i.hizliresim.com/mghuzni.png">

Kartlı geçiş sistemi örneğinde kart okunduğu zaman bir otokontrol gerçekleşir. Eğer kart tanımlıysa devam et ve kartın son hareketine bak, bu kişi içeride mi dışarıda mı? İçeride görünüyorsa geçişine izin verme gibi.

<img src = "https://i.hizliresim.com/3imu3cs.png">

Bunu yaparken de her seferinde WorkerTransactions tablosunu sıralayarak bir işlem gerçekleştirilir ve bu da totalde maliyetli hale gelir. Burada da son hareket bilgisi bir tabloda tutulursa daha kazançlı olur. 5-10 milyonluk tablo yerine her bir kaydın bir çalışan olduğu 2000 kayıtlık bir tablo üzerinde çalışırsın.

Aynı zamanda kimler içeride? kimler dışarıda? diye sorduğun zamanda da aşağıdaki işlemler gerçekleşmek durumundadır.

<img src = "https://i.hizliresim.com/alka3hn.png">

### Trigger ile Kayıtların Loglanması

<img src = "https://i.hizliresim.com/hato2ih.png">

# 2. Toplam Tablosu Oluşturma 1

### Hazırlık

* Triggerlar ile toplam hesaplama örneğini gerçekleştirmek için **TRG** databaseini oluştur.

* Bir **ITEMS** tablosu oluştur.

<img src = "https://i.hizliresim.com/r4tlz6q.png">

* **ITEMS** tablosu içerisine bir kaç kayıt at.

<img src = "https://i.hizliresim.com/dlr103g.png">

* Bir tane de stok hareketlerini gösteren **ITEMTRANSACTIONS** tablosu oluştur.

<img src = "https://i.hizliresim.com/ey4k04z.png">

* **ITEMTRANSACTIONS** tablosuna rastgele otomatik kayıt atacak bir script oluştur.
    * <code>RAND()</code> fonksiyonu ile 0 ile 1 arasında rastgele sayı üretebilirsin. 
    * Bu sayıyı bir tam sayı ile çarparsan o tam sayıya kadar rastgele bir ondalıklı sayı üretebilirsin.
    * <code>ROUND()</code> fonksiyonu ile de sonucu ondalıksız olarak yuvarlarsan istediğin sayı aralığında rastgele tamsayı üretebilirsin.
    * Bu sayede **ITEMID**, **AMOUNT**, **IOTYPE** kolonları rastgele bir sayı belirleyerek üretilebilir.
    * Eğer rastgele bir tarih üretmek istiyorsan da <code>DATEADD()</code> fonksiyonu ile son 1 yıl içerisi için 365 yazıp önüne - işareti getirerek sonuç alabilirsin.
    
<img src = "https://i.hizliresim.com/j6v1pu8.png">

* Daha fazla ürün eklemek için excelden yardım al.
    * Bir sütunun üç satırına 1, 2 ve 3 yazıp toplu seçersen hızlı çözümleme ile istediğin kadar satıra ardışık olarak sayı yazdırabilirsin.
    * Yan sütuna da = ile "" içerisinde URUN00 yazıp & ile hücreyi seçersen doldurma yapabilirsin. Her basamak artışında da bir 0 silersin.
    * Benzer şekilde yan sütuna da ürün adı için script yazıp *TRUNCATE* edilmiş **ITEMS** tablosuna Edit->Show SQL Pane'den kopyalarsın.
    
* Giriş çıkış kayıtlarını tek tek atmak yerine bir döngünün içerisinde istediğimiz kadar da atabiliriz.

<img src = "https://i.hizliresim.com/jwvefak.png">

* Bu sayede 1 milyondan fazla random veri elde edebiliriz.

<img src = "https://i.hizliresim.com/12d1und.png">

# 3. Toplam Tablosu Oluşturma 2

* Elimizdeki bu veriden toplam stoğu görmek istediğimizi düşünelim.

### Tek bir ürün için giriş-çıkış inceleme

<img src = "https://i.hizliresim.com/d8pkoo1.png">

* IOTYPE = 1 ise giriş, 2 ise çıkış olarak düşünelim.
* Buna göre ITEMID'si 1 olan ürün için;
    * Toplamda 8605 ürün için 1559 kez giriş işlemi,
    * Toplamda 8569 ürün için ise 1572 kez çıkış işlemi yapılmış.

### Her bir ürün için stok miktarını bulma

* İşlemlerdeki miktarların farkından eldeki stok öğrenilebilir.

<img src = "https://i.hizliresim.com/9oxqovv.png">

* Başına <code>SET STATISTICS IO ON</code> getirerek ne kadar okuma yaptığına baktığımızda aşağıdaki sonucu görürüz.

<img src = "https://i.hizliresim.com/4bu50ln.png">

* Tablo kolon bazında nispeten küçük olduğu için aslında bu okuma yeterlidir ancak daha fazla kolon okunsaydı sıkıntı olurdu.

* Bunun yerine elde ettiğimiz bu bilgi farklı bir tabloda (**STOCK**) tutuluyor olsaydı;

<img src = "https://i.hizliresim.com/n0d60up.png">

<img src = "https://i.hizliresim.com/2mpg4ak.png">

* Çıktıyı kopyalayıp **STOCK** tablomuza attığımızı düşünelim.

<img src = "https://i.hizliresim.com/kc3r4h8.png">

* Bu şekilde tablomuzu tuttuğumuzda çok daha hızlı okuma yapabiliriz.
* Artık **ITEMTRANSACTIONS** tablosuna yeni veriler geldiği takdirde **STOCK** tablosunu da yenilememizi sağlayacak trigger yapısını kurabiliriz.

# 4. Toplam Tablosu Insert Trigger

* **ITEMTRANSACTIONS** tablosunu ve stok değerlerini sıfırlayalım.

<img src = "https://i.hizliresim.com/af7x9ae.png">

* Şöyle bir yapı isteniyor.

<img src = "https://i.hizliresim.com/pig2gky.png">

* Yeni bir **ITENTRANSACTIONS** *INSERT*ünden sonra **STOCK** tablosuna ilgili ürün için de *UPDATE* işlemi gerçekleşsin.

* Mesela 10 ürün geldikten sonra 5 ürün gitsin tablolar böyle gözükür.

<img src = "https://i.hizliresim.com/rysw21r.png">

* Bizim manuel olarak yaptığımız bu işlemi sistemin otomatik olarak gerçekleştirmesini istiyoruz. 
    * Bunun da yolu tablonun altına bir **TRIGGER** yazmak.
    
### Trigger 

<img src = "https://i.hizliresim.com/lmvhoit.png">

* **TRIGGER**, **CREATE** ile isim verilerek oluşturulur.
* **ON** ile hangi tablo için oluşturulacağı seçilir.
* Insert işlemi sonrası çalışsın istediğimiz için **AFTER INSERT** diye belirtiriz.
* **AS** ve **BEGIN-END** ile blok açılır.
* Insert işlemi satır satır yapıldığı için Update işlemi de satır satır yapılır.
    * Buna göre her bir Insert işlemini *INSERTED* tablosundan alırız.
    * Bu tablodan ihtiyacımız olan ITEMID, AMOUNT ve IOTYPE kolonlarını atama yaparak alırız.
    * Giriş ve çıkış tiplerine göre **IF** bloklarını ayırarak STOCK tablosu üzerinde Update işlemini gerçekleştiririz.
    
<img src = "https://i.hizliresim.com/91g0a9z.png">  

* **TRIGGER** oluşturduktan sonra yaptığımız Insert işlemi artık iki satırı birden etkiler. 
    * *ITEMTRANSACTIONS* tablosuna yeni kayıt attıktan sonra bu kayıttan elde ettiği bilgileri *STOCK* tablosunda güncellemek için kullanır.
    
<img src = "https://i.hizliresim.com/7v7ppik.png">

* Daha önce yazdığımız rastgele veri ekleme scriptini çalıştırarak da **STOCK** tablosunun güncellendiğini görebiliriz.
    * Bu güncellemenin arka tarafta bir maliyeti vardır elbet ancak tablo üzerinde okuma yaptığımızda da 4 pagelik bir okuma gerçekleştiririz. Aksi takdirde aynı verinin atandığı durumda 8000 küsürlük page okuma gerçekleşirdi.

# 5. Toplam Tablosu Delete Trigger

* Peki **ITEMTRANSACTIONS** tablosundan kayıt silinirse **STOCK** tablosunda ne değişir?
    * Bunun için de bir **TRIGGER** oluşturulur ancak Insert işleminden sonra değil de Delete işleminden sonra çalışır.
    
<img src = "https://i.hizliresim.com/hss4r49.png">

* Insert Triggerdan farklı olarak **AFTER DELETE** ifadesi yer alır ve artık veriyi çektiğimiz tablo *DELETED* tablosudur.
* Silinen verilerden stokları güncelleyeceğimiz için de önceki Update işleminin tersi durumda bir işlem gerçekleşir.

<img src = "https://i.hizliresim.com/ol0xze4.png">

* ITEMID'si 1 olan ürünü incelediğimizde 5 adet transactions işlemi ve bu işlemler sonucu elimizde 8 adet ürün kaldığını görüyoruz.
* Eğer ITEMTRANSACTIONS tablosundaki 316 IDli kaydı silersek, bir çıkış işlemi olduğu için AMOUNTu kadar miktarın STOCK tablomuza eklendiğini görmeyi bekleriz.

<img src = "https://i.hizliresim.com/13dn9du.png">

* Öyle de olur.

<img src = "https://i.hizliresim.com/t9abmtf.png">

* Delete işlemi sonucunda da yine iki farklı tablonun birer satırlık değişikliklerini görebiliriz.

# 6. Toplam Tablosu Update Trigger

* Update sorgusu yazıldığında Stock tablosu nasıl güncellenir?

<img src = "https://i.hizliresim.com/ram2x8b.png">

* Update işlemi sonucunda sadece AMOUNT kolonlarının etkilendiğini varsayıyoruz.
* Update işlemi sonucunda hem Inserted hem de Deleted tablosu dolar.
* Deleted tablosunda eski miktar tutulur, Inserted tablosunda yeni miktar tutulur.
* **Eski değerden yeni değeri çıkarıyorsan Delete muamelesi yap, yeni değerden eski değeri çıkarıyorsan Insert muamelesi yap.**

Aşağıdaki ürünü inceleyelim.

<img src = "https://i.hizliresim.com/jm8b126.png">

* Buna göre yine her bir Update işleminde diğer işlemlerde olduğu gibi her iki tablo için de birer satırda işlem gerçekleşir ve STOCK tablosu Update işlemine uygun şekilde güncellenmeye devam eder.

# 7. Trigger ile Son Kaydı Loglama

* Trigger ile son kaydı aldırıp performans iyileştirmesi yapma.

### Hazırlık

* Öncelikle turnike örneğinde kullandığımız *WORKERS* tablosuna yeni veri ekleyelim. Bunu da elimizde bulunan CRM2 databaseindeki *CUSTOMERS* tablosundan uyarlayarak yapalım.

<img src = "https://i.hizliresim.com/l5f378v.png">

* Her bir müşteri için yıllık giriş çıkışlarını rastgele oluşturan procedureü döngü içerisinde her bir müşteri için çalıştıralım.

<img src = "https://i.hizliresim.com/bburzax.png">

* Atılan kayıtlar sonucu *WORKERTRANSACTIONS* tablomuz şu hale gelir.

<img src = "https://i.hizliresim.com/be68why.png">

* Herhangi bir çalışanın son hareketini bulalım.

<img src = "https://i.hizliresim.com/nzi62kl.png">

* Peki bu sonucu elde etmek için ne kadarlık bir page okumuş?

<img src = "https://i.hizliresim.com/qq238b8.png">

* Kim içeride kim dışarıda?

<img src = "https://i.hizliresim.com/nmhdc0s.png">

* Bu sonuç için ne kadarlık bir page okunmuş?

<img src = "https://i.hizliresim.com/c7gbrdi.png">

* **Bunun yerine her çalışanın son hareketini alan ve her bir son harekette güncellenen bir tablo olsaydı sorun çözülürdü.**

### WORKER_LAST_TRANSACTIONS

* İşlem gerçekleştiren çalışanın id bilgisini tutacak WORKERID, çalışanın işlem yaptığı son tarih bilgisini tutacak LASTDATE ve çalışanın son işlem türünü tutacak LASTIOTYPE kolonuna sahip tablomuzu tanımlayalım.

<img src= "https://i.hizliresim.com/5v3ciul.png">

* Sadece *WORKERS* tablosundan WORKERID Insert edilecek şekilde tablomuzu boş olarak dolduralım.

<img src = "https://i.hizliresim.com/f30d9hd.png">

### Trigger

<img src = "https://i.hizliresim.com/pijjyxg.png">

* Yeni hareket geleceği için **AFTER INSERT** komutu ve *INSERTED* tablosu kullanılarak veriler atanır ve güncelleme gerçekleştirilir.

<img src = "https://i.hizliresim.com/gvoqx9r.png">

* Procedure tekrar çağrıldığında artık trigger çalışır ve tablodaki son hareket bilgisi güncellenir haldedir.

<img src = "https://i.hizliresim.com/fbpigyi.png">

* Tekrardan döngüye alarak da *WORKER_LAST_TRANSACTIONS* tablomuzu otomatikmen doldurmaya başlarız.

### Page okuma

* İşlemi otomatikleştirmeden önce yaptığımız sorgu yaklaşık **5.8 milyon** pagelik bir okuma yaparken, aynı sonucu aldığımız trigger ile oluşturduğumuz tablomuz sorguladığımız takdirde **9** pagelik bir okuma yapar.

### Kullanım

* Bu örnek bir çok yerden (sensörlerde vs.) sürekli verilerin toplandığı IoT uygulamalarında sıklıkla kullanılır.

# 8. Trigger ile Loglama

* Triggerların bir diğer amacı da silinen, değiştirilen ve eklenen verileri kayıt altına almaktır.

### Hazırlık

* **ITEMS** tablosundaki herhangi bir ürünün fiyat değişimini tutacak bir tablo lazım.
    * Log tabloları tekrara müsait olduğu için ID alanını unique yapmana gerek yok.

* Değişiklik belli olsun diye gözlemlemek adına oluşturulan bu tablo (**ITEMS_LOG**) bir bakıma ITEMS tablosuna benzer kolonlar içerir.
* Ek olarak da ne değişiklik oldu, kim, ne zaman değiştirdi gibi bilgilerin de tutulması istenebilir.

<img src = "https://i.hizliresim.com/ifsjs5m.png">

* *LOG_ACTIONTYPE* : Yapılan değişikliğin türünü,
* *LOG_DATE* : Yapılan değişikliğin tarihini,
* *LOG_USERNAME* : Değişikliği yapan kişiyi,
* *LOG_PROGRAMNAME* : Değişikliğin yapıldığı program bilgisini,
* *LOG_HOSTNAME* : Değişikliğin yapıldığı bilgisayar bilgisini tutar.
* Üstteki diğer kolonlar ise değişikliğe uğrayabilecek kolonlardır.

### Update Trigger

<img src = "https://i.hizliresim.com/noaztgh.png">

* Birden fazla değişebilen kolon olabileceği için *INSERT INTO SELECT* yapısı kullanılmıştır.
* <code>SUSER_NAME()</code>: Sorguyu yazan kullanıcıyı verir.
* <code>PROGRAM_NAME()</code>: Sorgunun çağrıldığı programı verir.
* <code>HOST_NAME()</code>: Sorgunun yapıldığı bilgisayarı verir.

<img src = "https://i.hizliresim.com/c5s6izx.png">

* Herhangi bir değişiklikte hem *ITEMS* tablosu Update edildiği için satır etkilenir hem de *ITEMS_LOG* tablosuna yeni kayıt eklendiği için satır etkilenir.

<img src = "https://i.hizliresim.com/jsdp4r9.png">

* Değişiklikler her gerçekleştiğinde log tablosuna kayıtlar eklenmeye devam eder.

### Delete Trigger

<img src = "https://i.hizliresim.com/b0rleyj.png">

* Update Trigger'a benzer şekilde oluşturulur sadece Update ibareleri yerine Delete yazarsın. Deleted tablosunu kullanır.

<img src = "https://i.hizliresim.com/7bmbnj3.png">

* Artık ürünümüz *ITEMS* tablosundan silinmiştir ancak *ITEMS_LOG* tablosunda logları tutulmaya devam eder.

### Delete, Update Trigger

<img src = "https://i.hizliresim.com/8n180xk.png">

* *AFTER*'dan sonra **DELETE** ve **UPDATE** birlikte getirerek iki manipülasyon işlemini de triggerına dahil edebilirsin.
* Bu noktada yapılan işlemleri birbirlerinden ayırıp ona göre davranmak için *DELETED* ve *INSERTED* tablolarını kullanırsın.
* Eğer Update işlemi yapılmışsa hem *DELETED* hem de *INSERTED* tablosu dolar ancak Delete işlemi yapılmışsa sadece *DELETED* tablosu dolar.

<img src = "https://i.hizliresim.com/4a862kr.png">

* Artık her iki işlem için de tek bir triggerı kullanarak log tablonu doldurabilirsin.

# 9. Instead Of Trigger

* Instead Of Trigger'da veri manipülasyonunu yapılacak kişi manipüle edilir.
* Yani silme işlemi yapmak istediğini varsayarsak o silme işlemini yaptırmayıp başka bir işlem yaptırıp onun yaptırmak istediği silme işlemini de kayıt altına alabiliriz.
    
<img src = "https://i.hizliresim.com/fcyfa9d.png">

* Önceki derste oluşturulan triggerda *AFTER* yerine *INSTEAD OF* kullanırsak artık update ve delete işlemlerini gerçekleştirmez, o işlemler "yerine" triggerın içindekileri çalıştırır.

<img src = "https://i.hizliresim.com/s080geh.png">

* İşlemi yapan kullanıcıya karşı önlem alınır.
* Verimize bilerek ya da bilmeden silme işlemi yapılmasına müsade etmedik ve bu istenilen işlemi de kayıt altına aldık.